# Homework 1 – Initiation

### Tasks
1. Import provided `customers.xls` dataset to a pandas dataframe.
* Derive for all non-numeric columns one numeric feature.
* Standardize columns `age`, `est_income` and `local`.
* Derive four features by applying a PCA. Which of the original features has the highest correlation with one of the four derived features (principal components)?

### Preparation

In [8]:
# Import Python modules
import pandas as pd
import numpy as np
import xlrd 
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Import Data
data = pd.read_excel('data\customers.xls')

data

,ID,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,CHURN
0,1,F,S,1,38000.00,N,229.64,24.393333,3,23.56,0.00,206.08,0,CC,Budget,Intnl_discount,Cancelled
1,6,M,M,2,29616.00,N,75.29,49.426667,2,29.78,0.00,45.50,0,CH,FreeLocal,Standard,Current
2,8,M,M,0,19732.80,N,47.25,50.673333,3,24.81,0.00,22.44,0,CC,FreeLocal,Standard,Current
3,11,M,S,2,96.33,N,59.01,56.473333,1,26.13,0.00,32.88,1,CC,Budget,Standard,Current
4,14,F,M,2,52004.80,N,28.14,25.140000,1,5.03,0.00,23.11,0,CH,Budget,Intnl_discount,Cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,F,S,0,78851.30,N,29.04,48.373333,4,0.37,0.00,28.66,0,CC,FreeLocal,Standard,Cancelled
2066,3822,F,S,1,17540.70,Y,36.20,62.786667,1,22.17,0.57,13.45,0,Auto,Budget,Standard,Cancelled
2067,3823,F,M,0,83891.90,Y,74.40,61.020000,4,28.92,0.00,45.47,0,CH,Budget,Standard,Cancelled
2068,3824,F,M,2,28220.80,N,38.95,38.766667,4,26.49,0.00,12.46,0,CC,FreeLocal,Standard,Cancelled


In [9]:
data.describe()

,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped
count,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000
mean,1901.151691,1.147826,51464.262406,75.855063,42.798081,2.509662,16.129338,1.207295,59.080812,0.135749
std,1094.709118,0.843124,30796.680570,59.741635,14.883739,1.125157,9.866626,2.625421,57.542526,0.526189
min,1.000000,0.000000,96.330000,0.680000,12.326667,1.000000,0.000000,0.000000,0.680000,0.000000
25%,972.250000,0.000000,21084.700000,34.250000,30.886667,2.000000,8.090000,0.000000,15.200000,0.000000
50%,1874.000000,1.000000,55860.000000,57.110000,45.526667,2.000000,16.140000,0.000000,39.620000,0.000000
75%,2832.000000,2.000000,78000.000000,107.020000,54.013333,4.000000,22.990000,0.000000,87.460000,0.000000
max,3825.000000,2.000000,120000.000000,361.880000,77.000000,4.000000,59.000000,9.700000,332.460000,4.000000


### Task 1 | One-Hot-Encoding

In [10]:
# Replace all categorical features with discrete numeric values
data['Sex'].replace(to_replace=['M','F'], value=[0,1], inplace=True)
data['Status'].replace(to_replace=['S','M','D'], value=[0,1,2], inplace=True)
data['Car_Owner'].replace(to_replace=['N','Y'], value=[0,1], inplace=True)
data['Paymethod'].replace(to_replace=['CC', 'CH', 'Auto'], value=[0,1,2], inplace=True)
data['LocalBilltype'].replace(to_replace=['Budget','FreeLocal'], value=[0,1], inplace=True)
data['LongDistanceBilltype'].replace(to_replace=['Standard','Intnl_discount'], value=[0,1], inplace=True)
data['CHURN'].replace(to_replace=['Cancelled','Current'], value=[0,1], inplace=True)

data

,ID,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,CHURN
0,1,1,0,1,38000.00,0,229.64,24.393333,3,23.56,0.00,206.08,0,0,0,1,0
1,6,0,1,2,29616.00,0,75.29,49.426667,2,29.78,0.00,45.50,0,1,1,0,1
2,8,0,1,0,19732.80,0,47.25,50.673333,3,24.81,0.00,22.44,0,0,1,0,1
3,11,0,0,2,96.33,0,59.01,56.473333,1,26.13,0.00,32.88,1,0,0,0,1
4,14,1,1,2,52004.80,0,28.14,25.140000,1,5.03,0.00,23.11,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,1,0,0,78851.30,0,29.04,48.373333,4,0.37,0.00,28.66,0,0,1,0,0
2066,3822,1,0,1,17540.70,1,36.20,62.786667,1,22.17,0.57,13.45,0,2,0,0,0
2067,3823,1,1,0,83891.90,1,74.40,61.020000,4,28.92,0.00,45.47,0,1,0,0,0
2068,3824,1,1,2,28220.80,0,38.95,38.766667,4,26.49,0.00,12.46,0,0,1,0,0


### Task 2 | Standardization

In [12]:
# Standardize the features with z-score standardisation
data['Est_Income'] = StandardScaler().fit_transform(data['Est_Income'].to_numpy().reshape(-1,1))
data['Age'] = StandardScaler().fit_transform(data['Age'].to_numpy().reshape(-1,1))
data['Local'] = StandardScaler().fit_transform(data['Local'].to_numpy().reshape(-1,1))

data.drop(columns=['ID'], inplace=True)

data

,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,CHURN
0,1,0,1,-0.437304,0,229.64,-1.236866,3,23.56,0.00,2.555236,0,0,0,1,0
1,0,1,2,-0.709607,0,75.29,0.445465,2,29.78,0.00,-0.236071,0,1,1,0,1
2,0,1,0,-1.030602,0,47.25,0.529246,3,24.81,0.00,-0.636914,0,0,1,0,1
3,0,0,2,-1.668373,0,59.01,0.919027,1,26.13,0.00,-0.455440,1,0,0,0,1
4,1,1,2,0.017556,0,28.14,-1.186687,1,5.03,0.00,-0.625268,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,1,0,0,0.889500,0,29.04,0.374677,4,0.37,0.00,-0.528794,0,0,1,0,0
2066,1,0,1,-1.101799,1,36.20,1.343306,1,22.17,0.57,-0.793184,0,2,0,0,0
2067,1,1,0,1.053213,1,74.40,1.224580,4,28.92,0.00,-0.236592,0,1,0,0,0
2068,1,1,2,-0.754922,0,38.95,-0.270926,4,26.49,0.00,-0.810393,0,0,1,0,0


### Task 3 | PCA

In [13]:
# Build a PCA instance, 4 principal components
pca = PCA(n_components=4, random_state=42)

# Perform Singular Value Decomposition
data_pca = pd.DataFrame(data=pca.fit_transform(data), columns=['Feature_{}'.format(x) for x in range(1,5)])

data_pca

,Feature_1,Feature_2,Feature_3,Feature_4
0,153.980774,-1.099725,-1.514972,-0.060965
1,0.190710,13.692809,-0.728762,-0.528136
2,-28.083581,10.295222,-0.732333,0.396270
3,-16.266756,10.952951,-0.857931,-1.584000
4,-48.266957,-8.355633,-1.355214,-1.847349
...,...,...,...,...
2065,-47.626858,-13.065590,-1.527769,1.717410
2066,-39.264402,8.267089,-0.380306,-1.172357
2067,-0.746418,12.880863,-0.763910,2.176437
2068,-36.277200,12.456134,-0.626536,0.971072


In [14]:
# Analyse the percentages of explained variance

print('Explained Variance Ratio:')
pd.Series(pca.explained_variance_ratio_, index=['Feature_{}'.format(x) for x in range(1,5)])

Explained Variance Ratio:


Feature_1    0.972923
Feature_2    0.023527
Feature_3    0.001854
Feature_4    0.000353
dtype: float64

In [15]:
# Build a storage df
correlations = pd.DataFrame(index=data.columns, columns=data_pca.columns)

# For each feature, analyze the pearson correlation with each principal component
for feature in data.columns: 
    for feature_pca in data_pca.columns:
        correlations.loc[feature, feature_pca] = data[feature].corr(data_pca[feature_pca])

# For each feature, compute the maximum of the correlations with the principal components 
correlations['Max'] = abs(correlations.max(axis='columns'))

# Sort df to obtain the original feature with the highest correlation with one of the principal components
correlations.sort_values(by='Max', ascending=False)

,Feature_1,Feature_2,Feature_3,Feature_4,Max
Usage,0.999963,-0.00865001,-0.00018623,7.42989e-06,0.999963
International,0.0549133,-0.135453,0.988793,0.00135538,0.988793
Local,0.98697,-0.129055,-0.0486634,-0.00620524,0.986970
LongDistance,0.338853,0.940787,0.0098259,0.000303463,0.940787
RatePlan,0.0264598,0.0198806,0.0448023,0.902964,0.902964
Age,-0.0763444,-0.0856096,-0.191593,0.390167,0.390167
Est_Income,-0.0982616,-0.0507488,0.0340759,0.330927,0.330927
Sex,0.0315867,0.117651,-0.290998,0.015241,0.117651
LongDistanceBilltype,-0.0609154,-0.0408556,0.0637261,0.0062667,0.063726
Paymethod,-0.030809,0.0491195,0.00207759,0.0166897,0.049120
